In [1]:
import pandas as pd
import datetime as dt
import numpy as np
#import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier, BaggingRegressor, AdaBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from time import time
from nltk.corpus import stopwords

In [2]:
import re
import string
import nltk
from nltk.corpus import stopwords

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [4]:
from nltk.stem.snowball import SnowballStemmer

In [5]:
stemmer=SnowballStemmer(language='english')

In [6]:
def stemmizar_texto(texto):
    texto=' '.join([stemmer.stem(palabra) for palabra in texto.split() ])
    return texto

In [7]:
nltk.download('stopwords')
english_stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Santi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
train_set=pd.read_csv("train.csv",encoding='utf-8')
test_set=pd.read_csv("test.csv",encoding='utf-8')

In [9]:
def quitar_stopwords(texto):
    texto=' '.join([word for word in texto.split() if word not in english_stopwords])
    return texto

def quitar_menciones(texto):
    texto=' '.join([palabra for palabra in texto.split() if '@' not in palabra])
    return texto

def quitar_links(texto):
    texto=' '.join([palabra for palabra in texto.split() if 'http:/' not in palabra])
    return texto


In [10]:
Diccionario_de_lenguaje_de_internet={
"2moro": "Tomorrow",
"2nite": "Tonight",
"4EAE": "For Ever And Ever",
"ABT":"About",
"ADN": "Any Day Now",
"AFAIC": "As Far As I’m Concerned",
"AFAICT": "As Far As I Can Tell",
"AFAIK": "As Far As I Know",
"AFAIR": "As Far As I Remember",
"AKA": "Also Known As",
"AMA": "Ask Me Anything",
"ASAIC": "As Soon As I Can",
"ASAP": "As Soon As Possible",
"ATM": "At The Moment",
"B4": "Before",
"B4N": "Bye For Now",
"Bae": "Babe/Before Anyone Else",
"BBL": "Be Back Later",
"BBT": "Be Back Tomorrow",
"BCNU": "Be Seeing You",
"BD": "Big Deal",
"BF": "Boyfriend",
"BFF": "Best Friends Forever",
"BMT": "Before My Time",
"BOL": "Be On Later",
"BOT": "Back On Topic",
"BRB": "Be Right Back",
"BRO": "Brother",
"BT": "But",
"BTW": "By The Way",
"CFY": "Calling For You",
"CU": "See You",
"CUL": "See You Later",
"Cuz": "Because",
"CYA": "Cover Your Ass",
"DAE": "Does Anyone Else",
"DBA": "Doing Business As",
"DFTBA": "Don’t Forget To Be Awesome",
"DIKU": "Do I Know You",
"DM": "Direct Message",
"DND": "Do Not Disturb",
"DR": "Double Rainbow",
"DWBH": "Don’t Worry, Be Happy",
"ELI5": "Explain Like I’m 5",
"EOM": "End Of Message",
"EOS": "End Of Story",
"F2F": "Face To Face",
"FAQ": "Frequently Asked Question",
"FB": "Facebook",
"FBF": "Flash Back Friday",
"FF": "Follow Friday",
"FIFY": "Fixed It For You",
"FITB": "Fill In The Blank",
"FML": "Fuck My Life",
"FOMO": "Fear Of Missing Out",
"FTFY": "Fixed That For You",
"FTL": "For The Loss",
"FTW": "For The Win",
"FWB": "Friends With Benefits",
"FWIW": "For What It’s Worth",
"FYE": "For Your Entertainment",
"FYEO": "For Your Eyes Only",
"FYI": "For Your Information",
"GA": "Go Ahead",
"GAL": "Get A Life",
"GF": "Girlfriend",
"GM": "Good Morning",
"GN": "Good Night",
"Gr8": "Great",
"GTR": "Getting Ready",
"HAND": "Have A Nice Day",
"HB": "Hurry Back",
"HBD": "Happy Birthday",
"HBU": "How About You",
"HMB": "Hit Be Back",
"HMU": "Hit Me Up",
"HRU": "How Are You",
"HTH": "Hope This Helps",
"IAC": "In Any Case",
"IC": "I See",
"ICYMI": "In Case You Missed It",
"IDC": "I Don’t Care",
"IDK": "I Don’t Know",
"IG": "Instagram",
"IIRC": "If I Remember Correctly",
"IKR": "I Know Right",
"ILY": "I Love You",
"IMHO": "In My Humble Opinion",
"IMMD": "It Made My Day",
"IMY": "I Miss You",
"IRL": "In Real Life",
"IS": "I’m Sorry",
"ISO": "In Search Of",
"IU2U": "It’s Up To You",
"J4F": "Just For Fun",
"JAM": "Just A Minute",
"JFY": "Just For You",
"JIC": "Just In Case",
"JK": "Just Kidding",
"JSYK": "Just So You Know",
"KK": "Okay",
"L8": "Late",
"L8R": "Later",
"LMA": "Leave Me Alone",
"LMAO": "Laughing My Ass Off",
"LMBO": "Laughing My Butt Off",
"LMK": "Let Me Know",
"LOL": "Laugh Out Loud",
"LTNS": "Long Time No See",
"LYLAS": "Love You Like A Sister",
"M/F": "Male or Female",
"M8": "Mate",
"MP": "My pleasure",
"MSM": "Mainstream Media",
"MU": "Miss You",
"MYOB": "Mind Your Own Business",
"NAGI": "Not A Good Idea",
"NBD": "No Big Deal",
"NE1": "Anyone",
"NM": "Not Much",
"NP": "No Problem",
"NSFL": "Not Safe For Life",
"NSFW": "Not Safe For Work",
"NTS": "Note To Self",
"NVM": "Never Mind",
"OC": "Original Content",
"OH": "Overheard",
"OIC": "Oh ! I See",
"OMD": "Oh My Damn",
"OMG": "Oh My Goodness",
"OMW": "On My Way",
"OT": "Off Topic",
"OFC": "Of course",
"PAW": "Parents Are Watching",
"Pls": "Please",
"POTD": "Photo Of The Day",
"POV": "Point Of View",
"PPL": "People",
"PTB": "Please Text Back",
"Q4U": "Question For You",
"QQ": "Crying",
"RBTL": "Read Between The Lines",
"RIP": "Rest In Peace",
"RL": "Real Life",
"ROFL": "Rolling On the Floor Laughing",
"RT": "Retweet",
"RTM": "Read The Manual",
"SIS": "Sister",
"SITD": "Still In The Dark",
"SM": "Social Media",
"SMH": "Shaking My Head",
"SMY": "Somebody",
"SNH": "Sarcasm Noted Here",
"SOL": "Sooner Or Later",
"Some1": "Someone",
"SRSLY": "Seriously",
"STBY": "Sucks To Be You",
"Str8": "Straight",
"SYS": "See You Soon",
"TBA": "To Be Announced",
"TBH": "To Be Honest",
"TBT": "Throwback Thursday",
"TBT": "Truth Be Told",
"TFH": "Thread From Hell",
"TFTI": "Thanks For The Invite",
"TGIF": "Thank God It’s Friday",
"THX": "Thanks",
"TIA": "Thanks in Advance",
"TIL": "Today I Learned",
"TIME": "Tears In My Eyes",
"TL;DR": "Too Long; Didn’t Read",
"TLDR":"Too long didn’t read",
"TL DR":"Too long didn’t read",
"TLC": "Tender Loving Care",
"TMI": "Too Much Information",
"TTYL": "Talk To You Later",
"TTYS": "Talk To You Soon",
"Txt": "Text",
"TYVM": "Thank You Very Much",
"U": "You",
"U4F": "You Forever",
"UR": "Your",
"VBG": "Very Big Grin",
"VSF": "Very Sad Face",
"WB": "Welcome Back",
"WBU": "What About You?",
"WEG": "Wicked Evil Grin",
"WKND": "Weekend",
"WOM": "Word of Mouth",
"WOTD": "Word Of The Day",
"Wru": "Who Are You",
"WTH": "What The Heck?",
"WTPA": "Where The Party At?",
"WU?": "What's Up",
"WU":"What's Up",
"WYCM": "Will You Call Me?",
"WYWH": "Wish You Were Here",
"XOXO": "Hugs and Kisses",
"YGM": "You’ve Got Mail",
"YNK": "You Never Know",
"YOLO": "You Only Live Once",
"YT": "YouTube",
"YW": "You’re Welcome",
}

In [11]:
def reemplazar_lenguaje_internet(texto):
    texto=texto.upper()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=Diccionario_de_lenguaje_de_internet.get(palabra,'not internet slang')
        if(traduccion!='not internet slang'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [12]:
diccionario_contracciones = {  "ain't": "is not",
                               "aint":"is not",
                               "aren't": "are not",
                               "arent":"are not",
                               "can't": "can not",
                               "cant":"can not",
                               "cause": "because",
                               "could've": "could have",
                               "couldve":"could have",
                               "couldn't": "could not",
                               "couldnt":"could not",
                               "didn't": "did not", 
                               "didnt":"did not",
                               "doesn't": "does not",
                               "doesnt":"does not",
                               "don't": "do not",
                               "dont":"do not",
                               "hadn't": "had not",
                               "hadnt":"had not",
                               "hasn't": "has not",
                               "hasnt":"has not",
                               "haven't": "have not",
                               "havent":"have not",
                               "he'd": "he would",
                               "hed":"he would",
                               "he'll": "he will",
                               "he ll":"he will",
                               "he's": "he is",
                               "hes":"he is",
                               "how'd": "how did",
                               "howd":"how did",
                               "how'd'y": "how do you",
                               "howdy":"how do you",
                               "how'll": "how will",
                               "howll":"how will",
                               "how's": "how is",
                               "hows":"how is",
                               "I'd": "I would",
                               "id":"i would",
                               "I'd've": "I would have",
                               "idve":"i would have",
                               "I'll": "I will", 
                               "Ill":"i will", #duda: si algun tweet habla de un enfermo puede traer ruido
                               "I'll've": "I will have",
                               "I'm": "I am", 
                               "im":"i am",
                               "I've": "I have",
                               "ive":"i have",
                               "i'd": "i would",
                               "i'd've": "i would have",
                               "i'll": "i will",
                               "i'll've": "i will have",
                               "i'm": "i am", 
                               "im":"i am",
                               "i've": "i have",
                               "isn't": "is not",
                               "isnt":"is not",
                               "it'd": "it would",
                               "itd":"it would",
                               "it'd've": "it would have",
                               "it'll": "it will",
                               "itll":"it will",
                               "it'll've": "it will have",
                               "it's": "it is",
                               "its":"it is",
                               "let's": "let us",
                               "lets":"let us",
                               "ma'am": "madam",
                               "maam":"madam",
                               "mayn't": "may not",
                               "maynt":"may not",
                               "might've": "might have",
                               "mightve":"might have",
                               "mightn't": "might not",
                               "mightnt":"might not",
                               "mightn't've": "might not have",
                               "must've": "must have",
                               "mustve":"must have",
                               "mustn't": "must not",
                               "mustnt":"must not",
                               "mustn't've": "must not have",
                               "needn't": "need not",
                               "neednt":"need not",
                               "needn't've": "need not have",
                               "o'clock": "of the clock",
                               "oclock":"of the clock",
                               "oughtn't": "ought not",
                               "oughtnt":"ought not",
                               "oughtn't've": "ought not have",
                               "shan't": "shall not",
                               "shant":"shall not",
                               "sha'n't": "shall not",
                               "shan't've": "shall not have",
                               "she'd": "she would",
                               "shed":"she would",
                               "she'd've": "she would have",
                               "she'll": "she will",
                               "shell":"she will",#nuevamente aca tengo ruido por dos palabras igual escritas
                               "she'll've": "she will have", 
                               "she's": "she is",
                               "shes":"she is",
                               "should've": "should have",
                               "shouldve":"should have",
                               "shouldn't": "should not",
                               "shouldnt": "should not",
                               "shouldn't've": "should not have",
                               "so've": "so have",
                               "so's": "so as",
                               "this's": "this is",
                               "that'd": "that would",
                               "that'd've": "that would have",
                               "that's": "that is",
                               "thats":"that is",
                               "there'd": "there would",
                               "thered":"there would",
                               "there'd've": "there would have",
                               "there's": "there is",
                               "theres":"there is",
                               "here's": "here is",
                               "heres":"here is",
                               "they'd": "they would",
                               "theyd":"they would",
                               "they'd've": "they would have",
                               "they'll": "they will",
                               "theyll":"they will",
                               "they'll've": "they will have", 
                               "they're": "they are",
                               "theyre":"they are",
                               "they've": "they have", 
                               "theyve":"they have",
                               "to've": "to have", 
                               "tove":"to have",
                               "wasn't": "was not",
                               "wasnt":"was not",
                               "we'd": "we would",
                               "wed":"we would",#aca nuevamente hay conflicto
                               "we'd've": "we would have",
                               "we'll": "we will",
                               "we'll've": "we will have",
                               "we're": "we are",
                               "we've": "we have",
                               "weren't": "were not",
                               "werent":"were not",
                               "what'll": "what will",
                               "whatll":"what will",
                               "what'll've": "what will have",
                               "what're": "what are", 
                               "whatre":"what are",
                               "what's": "what is",
                               "whats":"what is",
                               "what've": "what have",
                               "whatve":"what have",
                               "when's": "when is",
                               "whens":"when is",
                               "when've": "when have",
                               "whenve":"when have",
                               "where'd": "where did",
                               "whered":"where did",
                               "where's": "where is",
                               "wheres":"where is",
                               "where've": "where have",
                               "whereve":"where have",
                               "who'll": "who will", 
                               "wholl":"who will",
                               "who'll've": "who will have",
                               "who's": "who is",
                               "whos":"who is",
                               "who've": "who have",
                               "whove":"who have",
                               "why's": "why is",
                               "whys":"why is",
                               "why've": "why have",
                               "whyve":"why have",
                               "will've": "will have",
                               "willve":"will have",
                               "won't": "will not",
                               "wont":"will not",
                               "won't've": "will not have",
                               "would've": "would have",
                               "wouldve":"would have",
                               "wouldn't": "would not",
                               "wouldnt":"would not",
                               "wouldn't've": "would not have",
                               "y'all": "you all",
                               "yall":"you all",
                               "y'all'd": "you all would",
                               "yalld":"you all would",
                               "y'all'd've": "you all would have",
                               "y'all're": "you all are",
                               "yallre":"you all are",
                               "y'all've": "you all have",
                               "you'd": "you would",
                               "youd":"you would",
                               "you'd've": "you would have",
                               "you'll": "you will",
                               "youll":"you will",
                               "you'll've": "you will have",
                               "you're": "you are",
                               "youre":"you are",
                               "you've": "you have",
                               "youve":"you have"}


In [13]:
def reemplazar_contracciones(texto):
    texto=texto.lower()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=diccionario_contracciones.get(palabra,'no es contraccion')
        if(traduccion!='no es contraccion'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [14]:
def eliminar_palabras_de_una_sola_letra(texto):
    texto=texto.lower()
    lista_palabras=texto.split()
    palabras_procesadas=[]
    for palabra in lista_palabras:
        if(len(palabra)!=1):
            palabras_procesadas.append(palabra)
    texto=" ".join([word for word in palabras_procesadas])
    return texto

In [15]:
#es necesario importar la libreria re (regular expression) para poder usar esta funcion
#ya esta bajada en el comienzo del notebook pero si lo que quieren es analizar la documentación
#lo pueden revisar en google
#falta borrar los arroba
def limpiar_texto(texto):
    texto = reemplazar_lenguaje_internet(texto)
    texto=reemplazar_contracciones(texto)
    texto=texto.lower()
    texto = re.sub('\[.*?\]', '', texto) # remove text in square brackets
    texto = re.sub('https?://\S+|www\.\S+', '', texto) # remove URLs
    texto = re.sub('<.*?>+', '', texto) # remove html tags
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # remove punctuation
    texto = re.sub('\n', '', texto) # remove words conatinaing numbers
    texto = re.sub('\w*\d\w*', '', texto)
    texto = re.sub('[‘’“”…]', '', texto)
    texto = quitar_stopwords(texto)
    texto = quitar_menciones(texto)
    texto = eliminar_palabras_de_una_sola_letra(texto)
    return texto

In [16]:
from nltk import WordNetLemmatizer

In [17]:
lemmatizer=WordNetLemmatizer()

In [18]:
def lemmatizar_texto(texto):
    lista_palabras=nltk.word_tokenize(texto)
    texto=' '.join([lemmatizer.lemmatize(word) for word in lista_palabras])
    return texto
#nota: la función esta definida aca, pero para ver los resultados que produce hay
#que ir al comienzo del notebook y ejecutar la celda de lemmatizar texto luego
#de haber limpiado el texto original

In [19]:
from nltk.corpus import wordnet

def get_wordnet_pos(palabra):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([palabra])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [20]:
def lemmatizar_con_pos(texto):
    lista_palabras=texto.split()
    texto=' '.join(lemmatizer.lemmatize(palabra,get_wordnet_pos(palabra)) for palabra in lista_palabras)
    return texto

### Feature: Cantidad de caracteres usados en tweets

In [21]:
cuenta_caract = train_set[['id','text']]
cuenta_caract['caracteres_usados'] = cuenta_caract['text'].str.len()
cuenta_caract_t = test_set[['id','text']]
cuenta_caract_t['caracteres_usados'] = cuenta_caract_t['text'].str.len()

<ipython-input-21-fa59aa1b33af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cuenta_caract['caracteres_usados'] = cuenta_caract['text'].str.len()
<ipython-input-21-fa59aa1b33af>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cuenta_caract_t['caracteres_usados'] = cuenta_caract_t['text'].str.len()


In [22]:
cuenta_caract = cuenta_caract[['id','caracteres_usados']]
cuenta_caract_t = cuenta_caract_t[['id','caracteres_usados']]

In [23]:
train_set=train_set.merge(cuenta_caract,on='id',how='left')

In [24]:
test_set=test_set.merge(cuenta_caract_t,on='id',how='left')

### Feature: cantidad de menciones por tweet

In [25]:
cant_menciones = train_set[['id','text']]
cant_menciones['menciones_realizadas']=cant_menciones['text'].str.count('@')
cant_menciones_t = test_set[['id','text']]
cant_menciones_t['menciones_realizadas']=cant_menciones_t['text'].str.count('@')

<ipython-input-25-17f2c8243a58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_menciones['menciones_realizadas']=cant_menciones['text'].str.count('@')
<ipython-input-25-17f2c8243a58>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_menciones_t['menciones_realizadas']=cant_menciones_t['text'].str.count('@')


In [26]:
cant_menciones = cant_menciones[['id','menciones_realizadas']]
cant_menciones_t=cant_menciones_t[['id','menciones_realizadas']]

In [27]:
train_set=train_set.merge(cant_menciones,on='id',how='left')

In [28]:
test_set=test_set.merge(cant_menciones_t,on='id',how='left')

### Feature: ID que comparten localización

In [29]:
tiene_localizacion = train_set[['id','location']]
tiene_localizacion_t = test_set[['id','location']]

In [30]:
tiene_localizacion['location'].loc[~(tiene_localizacion['location'].isnull())]=1
tiene_localizacion_t['location'].loc[~(tiene_localizacion_t['location'].isnull())]=1

C:\Users\Santi\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-30-17f4be06ad34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiene_localizacion['location'].loc[~(tiene_localizacion['location'].isnull())]=1
<ipython-input-30-17f4be06ad34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiene_localizacion_t['location'].lo

In [31]:
tiene_localizacion.fillna(0,inplace=True)
tiene_localizacion_t.fillna(0,inplace=True)

C:\Users\Santi\anaconda3\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [32]:
tiene_localizacion['location'].value_counts()

1    5080
0    2533
Name: location, dtype: int64

In [33]:
tiene_localizacion = tiene_localizacion.rename(columns={'location':'permite_location'})
tiene_localizacion_t = tiene_localizacion_t.rename(columns={'location':'permite_location'})

In [34]:
train_set=train_set.merge(tiene_localizacion,on='id',how='left')

In [35]:
test_set=test_set.merge(tiene_localizacion_t,on='id',how='left')

### Feature: usa Keyword

In [36]:
usa_keyword = train_set[['id','keyword']]
usa_keyword['keyword'].loc[~(usa_keyword['keyword'].isnull())] = 1
usa_keyword_t = test_set[['id','keyword']]
usa_keyword_t['keyword'].loc[~(usa_keyword_t['keyword'].isnull())] = 1

<ipython-input-36-1791a4cc735b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_keyword['keyword'].loc[~(usa_keyword['keyword'].isnull())] = 1
<ipython-input-36-1791a4cc735b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_keyword_t['keyword'].loc[~(usa_keyword_t['keyword'].isnull())] = 1


In [37]:
usa_keyword.fillna(0,inplace=True)
usa_keyword_t.fillna(0,inplace=True)

In [38]:
usa_keyword = usa_keyword.rename(columns={'keyword':'use_keyword'})
usa_keyword_t = usa_keyword_t.rename(columns={'keyword':'use_keyword'})

In [39]:
train_set = train_set.merge(usa_keyword,on='id',how='left')

In [40]:
test_set=test_set.merge(usa_keyword_t,on='id',how='left')

### Feature: Cita URL en Tweet

In [41]:
usa_url = train_set[['id','text']]
usa_url_t = test_set[['id','text']]

In [42]:
usa_url['cita_url'] = usa_url['text'].str.count('http')
usa_url_t['cita_url'] = usa_url_t['text'].str.count('http')

<ipython-input-42-e0ed60eeef45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_url['cita_url'] = usa_url['text'].str.count('http')
<ipython-input-42-e0ed60eeef45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_url_t['cita_url'] = usa_url_t['text'].str.count('http')


In [43]:
usa_url = usa_url[['id','cita_url']]
usa_url_t = usa_url_t[['id','cita_url']]

In [44]:
train_set = train_set.merge(usa_url,on='id',how='left')

In [45]:
test_set=test_set.merge(usa_url_t,on='id',how='left')

### Feature: usa Hashtag

In [46]:
usa_hashtag = train_set[['id','text']]
usa_hashtag['use_hashtag']=usa_hashtag['text'].str.count('#')
usa_hashtag_t = test_set[['id','text']]
usa_hashtag_t['use_hashtag']=usa_hashtag_t['text'].str.count('#')

<ipython-input-46-eef0326b5282>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_hashtag['use_hashtag']=usa_hashtag['text'].str.count('#')
<ipython-input-46-eef0326b5282>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_hashtag_t['use_hashtag']=usa_hashtag_t['text'].str.count('#')


In [47]:
usa_hashtag = usa_hashtag[['id','use_hashtag']]
usa_hashtag_t = usa_hashtag_t[['id','use_hashtag']]

In [48]:
train_set = train_set.merge(usa_hashtag,on='id',how='left')

In [49]:
test_set=test_set.merge(usa_hashtag_t,on='id',how='left')

### Feature: Cantidad de palabras

In [50]:
cant_palabras = train_set[['id', 'text']]
cant_palabras['cant_palabras'] = cant_palabras['text'].str.count(' ') + 1
cant_palabras_t = test_set[['id', 'text']]
cant_palabras_t['cant_palabras'] = cant_palabras_t['text'].str.count(' ') + 1

<ipython-input-50-9a582d2d4513>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_palabras['cant_palabras'] = cant_palabras['text'].str.count(' ') + 1
<ipython-input-50-9a582d2d4513>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_palabras_t['cant_palabras'] = cant_palabras_t['text'].str.count(' ') + 1


In [51]:
cant_palabras = cant_palabras[['id', 'cant_palabras']]
cant_palabras_t = cant_palabras_t[['id', 'cant_palabras']]

In [52]:
train_set = train_set.merge(cant_palabras,on='id',how='left')
test_set = test_set.merge(cant_palabras_t,on='id',how='left')

### Feature: Cantidad de abreviaciones de internet

In [53]:
def contar_abreviaciones(data):
    lista_cant = []
    for tweet in data:
        cant = 0
        tweet = quitar_stopwords(tweet)
        tweet = tweet.upper()
        tweet = tweet.split()
        for word in tweet:
            if (word in Diccionario_de_lenguaje_de_internet):
                cant = cant + 1
        lista_cant = lista_cant + [cant]
    return lista_cant

In [54]:
cant_abreviaciones = train_set[['id', 'text']]
cant_abreviaciones['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones['text'])
cant_abreviaciones_t = test_set[['id', 'text']]
cant_abreviaciones_t['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones_t['text'])

<ipython-input-54-1cd6de0a6bdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_abreviaciones['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones['text'])
<ipython-input-54-1cd6de0a6bdd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_abreviaciones_t['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones_t['text'])


In [55]:
cant_abreviaciones = cant_abreviaciones[['id', 'cant_abreviaciones']]
cant_abreviaciones_t = cant_abreviaciones_t[['id', 'cant_abreviaciones']]

In [56]:
train_set = train_set.merge(cant_abreviaciones,on='id',how='left')
test_set = test_set.merge(cant_abreviaciones_t,on='id',how='left')

### Feature: Location en Estados Unidos

In [57]:
lista_estados_usa = ["ALABAMA", "AL", "ALASKA", "AK", 
                     "ARIZONA", "AZ", "ARKANSAS", "AR", 
                     "CALIFORNIA", "CA", "COLORADO", "CO", 
                     "CONNECTICUT", "CT", "DELAWARE", "DE", 
                     "FLORIDA", "FL", "GEORGIA", "GA", 
                     "HAWAII", "HI", "IDAHO", "ID", 
                     "ILLINOIS", "IL", "INDIANA", "IN", 
                     "IOWA", "IA", "KANSAS", "KS", 
                     "KENTUCKY", "KY", "LOUISIANA", "LA", 
                     "MAINE", "ME", "MARYLAND", "MD", 
                     "MASSACHUSETTS", "MA", "MICHIGAN", "MI", 
                     "MINNESOTA", "MN", "MISSISSIPPI", "MS", 
                     "MISSOURI", "MO", "MONTANA", "MT", 
                     "NEBRASKA", "NE", "NEVADA", "NV", 
                     "NEW HAMPSHIRE", "NH", "NEW JERSEY", 
                     "NJ", "NEW MEXICO", "NM", "NEW YORK", 
                     "NY", "NORTH CAROLINA", "NC", "NORTH DAKOTA", 
                     "ND", "OHIO", "OH", "OKLAHOMA", "OK", 
                     "OREGON", "OR", "PENNSYLVANIA", "PA", 
                     "RHODE ISLAND", "RI", "SOUTH CAROLINA", "SC", "CAROLINA", 
                     "SOUTH DAKOTA", "SD", "TENNESSEE", "TN", 
                     "TEXAS", "TX", "UTAH", "UT", 
                     "VERMONT", "VT", "VIRGINIA", "VA", 
                     "WASHINGTON", "WA", "WEST VIRGINIA", "WV", 
                     "WISCONSIN", "WI", "WYOMING", "WY",
                     "USA", "UNITED STATES", "SAN FRANCISCO"]

In [58]:
def ver_location_usa(data):
    lista_ubicacion_usa = []
    for location in data:
        if (location is None):
            lista_ubicacion_usa += [0]
        else:
            location_usa = False
            location = str(location)
            location = location.upper()
            if (location in lista_estados_usa):
                location_usa = True
            location = location.split()
            for word in location:
                if (word in lista_estados_usa):
                    location_usa = True
                    break
            if (location_usa):
                lista_ubicacion_usa += [1]
            else:
                lista_ubicacion_usa += [0]
    return lista_ubicacion_usa

In [59]:
location_usa = train_set[['id', 'location']]
location_usa['location_usa'] = ver_location_usa(location_usa['location'])
location_usa_t = test_set[['id', 'location']]
location_usa_t['location_usa'] = ver_location_usa(location_usa_t['location'])

<ipython-input-59-aa4731089d51>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_usa['location_usa'] = ver_location_usa(location_usa['location'])
<ipython-input-59-aa4731089d51>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_usa_t['location_usa'] = ver_location_usa(location_usa_t['location'])


In [60]:
location_usa = location_usa[['id', 'location_usa']]
location_usa_t = location_usa_t[['id', 'location_usa']]

In [61]:
train_set = train_set.merge(location_usa,on='id',how='left')
test_set = test_set.merge(location_usa_t,on='id',how='left')

### Feature: Tweet tiene caritas

In [62]:
lista_caritas = [":)", ";)", ":(", ";(", "XD", "xD", "xd", ":P", ":p", "-_-", ":O", ":'(", ":D", ":-D"]

In [63]:
def buscar_caritas(data):
    tiene_carita = []
    for tweet in data:
        tweet = quitar_menciones(tweet)
        tweet = quitar_links(tweet)
        hay_carita = False
        for carita in lista_caritas:
            if (carita in tweet):
                hay_carita = True
                break
        if (hay_carita):
            tiene_carita += [1]
        else:
            tiene_carita += [0]
    return tiene_carita

In [64]:
has_emoji = train_set[['id', 'text']]
has_emoji['has_emoji'] = buscar_caritas(has_emoji['text'])
has_emoji_t = test_set[['id', 'text']]
has_emoji_t['has_emoji'] = buscar_caritas(has_emoji_t['text'])

NameError: name 'quitar_links' is not defined

In [ ]:
has_emoji = has_emoji[['id', 'has_emoji']]
has_emoji_t = has_emoji_t[['id', 'has_emoji']]

In [ ]:
train_set = train_set.merge(has_emoji,on='id',how='left')
test_set = test_set.merge(has_emoji_t,on='id',how='left')

### Feature: Palabras con misma letra repetida

In [ ]:
lista_letras_seguidas = ["aaa", "bbb", "ccc", "ddd", "eee", "fff", "ggg", "hhh", "iii", "jjj", "kkk", "lll", "mmm", "nnn", "ooo", "ppp", "qqq", "rrr", "sss", "ttt", "uuu", "vvv", "www", "xxx", "yyy", "zzz"]

In [ ]:
def buscar_letras_seguidas(data):
    tiene_letras_seguidas = []
    for tweet in data:
        tweet = quitar_menciones(tweet)
        tweet = quitar_links(tweet)
        hay_letra_seguida = False
        for letra in lista_letras_seguidas:
            if (letra in tweet):
                hay_letra_seguida = True
                break
        if (hay_letra_seguida):
            tiene_letras_seguidas += [1]
        else:
            tiene_letras_seguidas += [0]
    return tiene_letras_seguidas

In [ ]:
has_repeated_letter = train_set[['id', 'text']]
has_repeated_letter['has_repeated_letter'] = buscar_letras_seguidas(has_repeated_letter['text'])
has_repeated_letter_t = test_set[['id', 'text']]
has_repeated_letter_t['has_repeated_letter'] = buscar_letras_seguidas(has_repeated_letter_t['text'])

In [ ]:
has_repeated_letter = has_repeated_letter[['id', 'has_repeated_letter']]
has_repeated_letter_t = has_repeated_letter_t[['id', 'has_repeated_letter']]

In [ ]:
train_set = train_set.merge(has_repeated_letter,on='id',how='left')
test_set = test_set.merge(has_repeated_letter_t,on='id',how='left')

In [ ]:
train_set.to_csv('train_data.csv')

In [ ]:
test_set.to_csv('test_data.csv')

In [ ]:
train_set

In [ ]:
test_set

## Feature: cantidad de palabras únicas usadas

In [ ]:
train_set=pd.read_csv("train_data.csv")

In [ ]:
lista_cantidad_de_palabras_unicas=[]
def contar_cantidad_de_palabras_unicas(texto):
    diccionario_auxiliar={}
    palabras=texto.split()
    for palabra in palabras:
        esta=diccionario_auxiliar.get(palabra,"no esta")
        if(esta=="no esta"):
            diccionario_auxiliar[palabra]="esta"
    lista_cantidad_de_palabras_unicas.append(len(diccionario_auxiliar))

In [ ]:
train_set['numero_de_palabras_unicas']=train_set['text'].apply

## Feature: Hace mencion a alguna ciudad o país

In [ ]:
ciudades_del_mundo=pd.read_csv('world-cities_csv.csv')

In [ ]:
ciudades_del_mundo.head()

In [ ]:
LISTA_CIUDADES=[]
def generar_lista_ciudades(texto):
    texto=str(texto)
    texto=texto.lower()
    lista_auxiliar=texto.split()
    if len(lista_auxiliar)!=1:
        texto=' '.join([word for word in lista_auxiliar])
    global LISTA_CIUDADES
    LISTA_CIUDADES.append(texto)

In [ ]:
LISTA_CIUDADES

In [ ]:
ciudades=ciudades_del_mundo['name']

In [ ]:
ciudades=list(ciudades)

In [ ]:
len(ciudades)

In [ ]:
def menciona_ciudad(texto):
    texto=texto.lower()
    global ciudades
    for ciudad in ciudades:
        lista_auxiliar=ciudad.split()
        if len(lista_auxiliar)!=1:
            ciudad=' '.join([word for word in lista_auxiliar])
        if ciudad.lower() in texto:
            return ciudad
    return None

In [ ]:
def ciudades_por_longitud(lista_ciudades):
    diccionario={}
    for ciudad in lista_ciudades:
        longitud=len(ciudad)
        contador=diccionario.get(longitud,-1)
        if contador==-1:
            diccionario[longitud]=1
        else:
            diccionario[longitud]=contador+1
    for key in diccionario.keys():
        print("llave: " + str(key) + "   "+ str(diccionario[key]))

In [ ]:
#ciudades_por_longitud(ciudades)

In [ ]:
train_set=pd.read_csv('train_data.csv')

In [ ]:
train_set['menciona_ciudad']=train_set['text'].apply(menciona_ciudad)

In [ ]:
train_set.loc[train_set['menciona_ciudad']!=None]

In [ ]:
dicc_paises={}
def generar_lista_paises(texto):
    global dicc_paises
    texto=texto.lower()
    esta=dicc_paises.get(texto,-1)
    if esta==-1:
        dicc_paises[texto]=1


In [ ]:
lista_paises=dicc_paises.keys()
def menciona_pais(texto):
    texto=texto.lower()
    for pais in lista_paises:
        if pais in texto:
            return 1
    return 0

In [ ]:
ciudades_del_mundo['country'].apply(generar_lista_paises)

In [ ]:
train_set['menciona_pais']=train_set['text'].apply(menciona_pais)

In [ ]:
train_set.head()

# Las siguientes features usarlas sólo despues de limpiar el texto

In [ ]:
train_set['text']=train_set['text'].apply(limpiar_texto)

In [ ]:
train_set['text']=train_set['text'].apply(lemmatizar_con_pos)

In [ ]:
train_set['text']=train_set['text'].apply(limpiar_texto)

### Feature: cantidad de palabras en el tweet

In [ ]:
def contar_cantidad_de_palabras(texto):
    contador=0
    texto=texto.lower()
    lista_palabras=texto.split()
    for palabra in lista_palabras:
        contador+=1
    return contador

In [ ]:
train_set['numero_de_palabras']=train_set['text'].apply(contar_cantidad_de_palabras)

### Feature: cantidad de palabras únicas utilizadas


In [ ]:
# nota: aplicar la función solamente una vez que el texto fue limpiado
def contar_cantidad_de_palabras_unicas(texto):
    diccionario_auxiliar={}
    palabras=texto.split()
    for palabra in palabras:
        esta=diccionario_auxiliar.get(palabra,"no esta")
        if(esta=="no esta"):
            diccionario_auxiliar[palabra]="esta"
    return(len(diccionario_auxiliar))

In [ ]:
train_set['palabras_unicas']=train_set['text'].apply(contar_cantidad_de_palabras_unicas)

### Feature: cantidad de palabras no reconocidas

In [ ]:
path_glove='/home/tomas/organizacion_de_datos/tp2/glove.6B.100d.txt'

In [ ]:
diccionario_de_palabras={}
def generar_diccionario_de_palabras():
    embeddings=open(path_glove,'r',encoding='UTF-8')
    while(True):
        f=embeddings.readline()
        if(len(f)==0):
            break
        f=f.split()
        lista=f[0].split()
        diccionario_de_palabras[lista[0]]='y'
    embeddings.close()

In [ ]:
generar_diccionario_de_palabras()

In [ ]:
def contar_palabras_desconocidas(texto):
    global diccionario_de_palabras
    contador=0
    texto=texto.lower()
    lista_palabras=texto.split()
    for palabra in lista_palabras:
        esta=diccionario_de_palabras.get(palabra,'no esta')
        if esta=='no esta':
            contador+=1
    return contador

In [ ]:
train_set['numero_palabras_desconocidas']=train_set['text'].apply(contar_palabras_desconocidas)

In [ ]:
train_set.head()

In [ ]:
test_set=pd.read_csv('test_data.csv')

In [ ]:
test_set['menciona_pais']=test_set['text'].apply(menciona_pais)

In [ ]:
test_set['text']=test_set['text'].apply(limpiar_texto)

In [ ]:
test_set['text']=test_set['text'].apply(lemmatizar_con_pos)

In [ ]:
test_set['text']=test_set['text'].apply(limpiar_texto)

In [ ]:
test_set['numero_de_palabras']=test_set['text'].apply(contar_cantidad_de_palabras)

In [ ]:
test_set['palabras_unicas']=test_set['text'].apply(contar_cantidad_de_palabras_unicas)

In [ ]:
test_set['numero_palabras_desconocidas']=test_set['text'].apply(contar_palabras_desconocidas)

In [ ]:
def convertir(texto):
    if texto=="Si":
        return 1
    return 0

In [ ]:
test_set['menciona_pais']=test_set['menciona_pais'].apply(convertir)

In [ ]:
test_set